# **Real-ESRGAN Standalone Upscaler (Fast Mode)**

A fast upscaling solution using Real-ESRGAN/ESRGAN models directly without diffusion.

## Speed Comparison
- **This notebook**: ~10-60 seconds per image (4x upscale)
- **FLUX notebook**: ~10-20 minutes per image (4x upscale)

## When to Use
- Fast batch processing
- When original image quality is already good
- Preview/proof before final FLUX upscale
- When you don't need AI-generated detail enhancement

## Available Models
- `4x-UltraSharp.pth` - Sharp details, good for digital art (default)
- `4x_foolhardy_Remacri.pth` - Natural textures, less over-sharpening
- `4x-AnimeSharp.pth` - Optimized for anime/illustration

In [8]:
# @title Setup Environment (Run Once)
import subprocess
import sys
import os
from pathlib import Path

# Get repo root
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

# Directory paths
HF_DIR = REPO_ROOT / "hf"
MODELS_DIR = HF_DIR / "models"
UPSCALE_MODELS_DIR = MODELS_DIR / "upscale_models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"

# Create directories
for d in [HF_DIR, MODELS_DIR, UPSCALE_MODELS_DIR, OUTPUT_DIR, INPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Models directory: {UPSCALE_MODELS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

def install_packages():
    """Install required packages for Real-ESRGAN standalone."""
    # PyTorch with CUDA 12.8 for RTX 5060 Ti
    print("Installing PyTorch nightly with CUDA 12.8...")
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-q', '--pre',
         'torch', 'torchvision',
         '--index-url', 'https://download.pytorch.org/whl/nightly/cu128'],
        capture_output=True
    )
    if result.returncode == 0:
        print("✓ PyTorch installed")
    else:
        print(f"✗ PyTorch install failed: {result.stderr.decode()}")
    
    # Spandrel for loading upscale models
    packages = ['spandrel', 'opencv-python', 'huggingface_hub', 'safetensors', 'pillow']
    for package in packages:
        try:
            subprocess.run(
                [sys.executable, '-m', 'pip', 'install', '-q', package],
                check=True, capture_output=True
            )
            print(f"✓ {package} installed")
        except subprocess.CalledProcessError as e:
            print(f"✗ Error installing {package}")

print("Installing packages...")
install_packages()
print("\n✅ Environment Setup Complete!")

Repo root: C:\Users\Armaan\Desktop\Artinafti
Models directory: C:\Users\Armaan\Desktop\Artinafti\hf\models\upscale_models
Output directory: C:\Users\Armaan\Desktop\Artinafti\4xoutput
Installing packages...
Installing PyTorch nightly with CUDA 12.8...
✓ PyTorch installed
✓ spandrel installed
✓ opencv-python installed
✓ huggingface_hub installed
✓ safetensors installed
✓ pillow installed

✅ Environment Setup Complete!


In [9]:
# @title Download Models (Run Once)
from huggingface_hub import hf_hub_download
from pathlib import Path
import os

# Ensure paths are set
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR
UPSCALE_MODELS_DIR = REPO_ROOT / "hf" / "models" / "upscale_models"
UPSCALE_MODELS_DIR.mkdir(parents=True, exist_ok=True)

def download_model(repo_id: str, filename: str, dest_dir: Path) -> str:
    """Download model from HuggingFace Hub."""
    dest_path = dest_dir / filename
    if dest_path.exists():
        print(f"✓ {filename} already exists")
        return filename
    
    try:
        print(f"Downloading {filename}...", end=' ', flush=True)
        hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            local_dir=str(dest_dir),
            local_dir_use_symlinks=False
        )
        print("Done!")
        return filename
    except Exception as e:
        print(f"\nError downloading {filename}: {e}")
        return None

print("Downloading upscale models...")
download_model("Isi99999/Upscalers", "4x-UltraSharp.pth", UPSCALE_MODELS_DIR)
download_model("Isi99999/Upscalers", "4x_foolhardy_Remacri.pth", UPSCALE_MODELS_DIR)
download_model("Isi99999/Upscalers", "4x-AnimeSharp.pth", UPSCALE_MODELS_DIR)

print("\n✅ Models downloaded!")

✓ 4x-UltraSharp.pth already exists
✓ 4x_foolhardy_Remacri.pth already exists
✓ 4x-AnimeSharp.pth already exists

✅ Models downloaded!


In [10]:
# @title Load Libraries
import torch
import numpy as np
from PIL import Image
import cv2
import gc
import os
import time
from pathlib import Path
from IPython.display import display, Image as IPImage

# Import spandrel for model loading
from spandrel import ImageModelDescriptor, ModelLoader

# Set up paths
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

UPSCALE_MODELS_DIR = REPO_ROOT / "hf" / "models" / "upscale_models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Device setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def clear_memory():
    """Clear GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

print(f"✅ Libraries loaded!")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

✅ Libraries loaded!
CUDA available: True
GPU: NVIDIA GeForce RTX 5060 Ti
VRAM: 15.9 GB


In [11]:
# @title Configuration - 6x Upscale (Single Image, No Downsizing)

INPUT_IMAGE = str(REPO_ROOT / "9ab756e9-8505-48bd-aa38-3c360d1e29b6.jpg")
OUTPUT_DIR = REPO_ROOT / "6x"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_FORMATS = ["png", "tiff"]

# ============== UPSCALE SETTINGS ==============
# Model options: "4x-UltraSharp.pth", "4x_foolhardy_Remacri.pth", "4x-AnimeSharp.pth"
upscale_model = "4x-UltraSharp.pth"

# Tile size for processing
tile_size = 512
tile_overlap = 32

# Use FP16 for faster processing
use_fp16 = True

# Two-pass: 4x model twice (16x total) then Lanczos down to 6x for best quality
USE_TWO_PASS = True

print("Configuration set!")
print(f"Input: {INPUT_IMAGE}")
print(f"Output dir: {OUTPUT_DIR}")
print(f"Mode: Pure 6x upscale (two-pass 16x -> Lanczos to 6x, no downsizing)")

Configuration set!
Input: C:\Users\Armaan\Desktop\Artinafti\9ab756e9-8505-48bd-aa38-3c360d1e29b6.jpg
Output dir: C:\Users\Armaan\Desktop\Artinafti\6x
Mode: Pure 6x upscale (two-pass 16x -> Lanczos to 6x, no downsizing)


In [12]:
# @title Upscale Functions

def upscale_with_tiles(model, img_tensor: torch.Tensor, tile_size: int, tile_overlap: int) -> torch.Tensor:
    """
    Upscale image using tiled processing to handle large images.
    
    Args:
        model: Loaded spandrel model
        img_tensor: Input image tensor (B, C, H, W)
        tile_size: Size of each tile
        tile_overlap: Overlap between tiles
    
    Returns:
        Upscaled image tensor
    """
    scale = model.scale
    _, _, h, w = img_tensor.shape
    
    # If image is small enough, process directly
    if h <= tile_size and w <= tile_size:
        with torch.no_grad():
            return model(img_tensor)
    
    # Calculate output size
    out_h, out_w = h * scale, w * scale
    output = torch.zeros((1, 3, out_h, out_w), device=img_tensor.device, dtype=img_tensor.dtype)
    weight = torch.zeros((1, 1, out_h, out_w), device=img_tensor.device, dtype=img_tensor.dtype)
    
    # Calculate tile positions
    stride = tile_size - tile_overlap
    h_tiles = max(1, (h - tile_overlap) // stride + (1 if (h - tile_overlap) % stride else 0))
    w_tiles = max(1, (w - tile_overlap) // stride + (1 if (w - tile_overlap) % stride else 0))
    
    total_tiles = h_tiles * w_tiles
    print(f"Processing {total_tiles} tiles ({h_tiles}x{w_tiles})...")
    
    tile_count = 0
    for i in range(h_tiles):
        for j in range(w_tiles):
            # Calculate tile boundaries
            y1 = min(i * stride, h - tile_size) if h > tile_size else 0
            x1 = min(j * stride, w - tile_size) if w > tile_size else 0
            y2 = min(y1 + tile_size, h)
            x2 = min(x1 + tile_size, w)
            
            # Extract and process tile
            tile = img_tensor[:, :, y1:y2, x1:x2]
            
            with torch.no_grad():
                tile_out = model(tile)
            
            # Calculate output positions
            out_y1, out_y2 = y1 * scale, y2 * scale
            out_x1, out_x2 = x1 * scale, x2 * scale
            
            # Create weight mask for blending
            tile_h, tile_w = tile_out.shape[2:]
            mask = torch.ones((1, 1, tile_h, tile_w), device=tile_out.device, dtype=tile_out.dtype)
            
            # Feather edges for blending
            feather = tile_overlap * scale // 2
            if feather > 0:
                # Top edge
                if i > 0:
                    for k in range(feather):
                        mask[:, :, k, :] *= k / feather
                # Bottom edge
                if i < h_tiles - 1:
                    for k in range(feather):
                        mask[:, :, -(k+1), :] *= k / feather
                # Left edge
                if j > 0:
                    for k in range(feather):
                        mask[:, :, :, k] *= k / feather
                # Right edge
                if j < w_tiles - 1:
                    for k in range(feather):
                        mask[:, :, :, -(k+1)] *= k / feather
            
            # Add to output with blending
            output[:, :, out_y1:out_y2, out_x1:out_x2] += tile_out * mask
            weight[:, :, out_y1:out_y2, out_x1:out_x2] += mask
            
            tile_count += 1
            print(f"  Processed {tile_count}/{total_tiles} tiles", end='\r')
    
    print(f"  Processed {tile_count}/{total_tiles} tiles")
    
    # Normalize by weights
    output = output / weight.clamp(min=1e-8)
    
    return output


def save_image_formats(img_array: np.ndarray, output_name: str, output_dir: Path, formats: list) -> list:
    """Save image in multiple formats.
    
    Args:
        img_array: Image as numpy array (H, W, C) in RGB format, uint8
        output_name: Base filename without extension
        output_dir: Output directory
        formats: List of formats to save, e.g. ["png", "tiff"]
    
    Returns:
        List of saved file paths
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    pil_img = Image.fromarray(img_array)
    
    saved_paths = []
    for fmt in formats:
        fmt_lower = fmt.lower()
        if fmt_lower == "png":
            output_path = output_dir / f"{output_name}.png"
            pil_img.save(str(output_path), "PNG")
            saved_paths.append(str(output_path))
            print(f"   Saved: {output_path.name}")
        elif fmt_lower in ["tiff", "tif"]:
            output_path = output_dir / f"{output_name}.tiff"
            pil_img.save(str(output_path), "TIFF", compression=None)
            saved_paths.append(str(output_path))
            print(f"   Saved: {output_path.name}")
    
    return saved_paths


def upscale_and_resize(
    image_path: str,
    target_width_inches: float,
    target_height_inches: float,
    dpi: int,
    output_name: str,
    output_formats: list,
    model_name: str,
    tile_size: int,
    tile_overlap: int,
    use_fp16: bool,
    use_two_pass: bool
) -> dict:
    """
    Upscale image with Real-ESRGAN using aspect-ratio-aware scaling.
    
    The output will be sized so that after human cropping to the target
    aspect ratio, the image will have exactly the target DPI.
    
    Args:
        image_path: Path to input image
        target_width_inches: Target print width in inches
        target_height_inches: Target print height in inches
        dpi: Target DPI for printing
        output_name: Output filename (without extension)
        output_formats: List of formats to save, e.g. ["png", "tiff"]
        model_name: Name of upscale model
        tile_size: Tile size for processing
        tile_overlap: Overlap between tiles
        use_fp16: Use FP16 precision
        use_two_pass: If True, run 4x model twice for 16x total
    
    Returns:
        dict with output paths and crop info
    """
    start_time = time.time()
    
    # Resolve image path
    image_path = Path(image_path)
    if not image_path.is_absolute():
        image_path = REPO_ROOT / image_path
    
    if not image_path.exists():
        raise FileNotFoundError(f"Input file not found: {image_path}")
    
    print(f"Processing: {image_path.name}")
    print(f"Target print size: {target_width_inches}\" x {target_height_inches}\" @ {dpi} DPI")
    
    # Load image
    print("Loading image...")
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    input_aspect = w / h
    target_aspect = target_width_inches / target_height_inches
    print(f"Input size: {w}x{h} (aspect: {input_aspect:.3f})")
    print(f"Target aspect: {target_aspect:.3f}")
    
    # Calculate aspect-ratio-aware output dimensions
    scale_info = calculate_scale_for_crop(
        input_width=w,
        input_height=h,
        target_width_inches=target_width_inches,
        target_height_inches=target_height_inches,
        dpi=dpi
    )
    
    output_width = scale_info["output_width_px"]
    output_height = scale_info["output_height_px"]
    final_width = scale_info["final_width_px"]
    final_height = scale_info["final_height_px"]
    
    print(f"\n📐 Scaling Strategy:")
    print(f"   Pre-crop output: {output_width}x{output_height}px ({scale_info['equivalent_print_size']})")
    print(f"   Final after crop: {final_width}x{final_height}px ({target_width_inches}\" x {target_height_inches}\")")
    if scale_info["crop_direction"] != "none":
        print(f"   Crop needed: {scale_info['crop_amount_px']}px {scale_info['crop_direction']} ({scale_info['crop_amount_inches']:.2f}\")")
    
    # Calculate effective scale after upscaling
    effective_scale = 16 if use_two_pass else 4
    upscaled_w, upscaled_h = w * effective_scale, h * effective_scale
    
    if use_two_pass:
        print(f"\n   Step 1: Two-pass 4x upscale (16x total) -> {upscaled_w}x{upscaled_h}")
    else:
        print(f"\n   Step 1: 4x upscale -> {upscaled_w}x{upscaled_h}")
    print(f"   Step 2: Resize to pre-crop size -> {output_width}x{output_height}")
    
    # Convert to tensor
    img_tensor = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    img_tensor = img_tensor.to(DEVICE)
    
    if use_fp16 and DEVICE.type == "cuda":
        img_tensor = img_tensor.half()
    
    # Load model
    print(f"\nLoading model: {model_name}")
    model_path = UPSCALE_MODELS_DIR / model_name
    if not model_path.exists():
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    model = ModelLoader().load_from_file(str(model_path))
    assert isinstance(model, ImageModelDescriptor), "Not an image model!"
    
    model = model.to(DEVICE)
    if use_fp16 and DEVICE.type == "cuda":
        model = model.half()
    model.eval()
    
    scale = model.scale
    print(f"Model scale: {scale}x")
    
    # First pass upscale
    print("Upscaling (pass 1 of {})...".format(2 if use_two_pass else 1))
    upscale_start = time.time()
    
    output_tensor = upscale_with_tiles(model, img_tensor, tile_size, tile_overlap)
    
    pass1_time = time.time() - upscale_start
    print(f"Pass 1 took: {pass1_time:.1f}s")
    
    # Second pass if requested
    if use_two_pass:
        print("Upscaling (pass 2 of 2)...")
        pass2_start = time.time()
        
        # Use smaller tile size for second pass (image is now 4x larger)
        tile_size_pass2 = min(tile_size, 384)  # Smaller tiles for larger image
        output_tensor = upscale_with_tiles(model, output_tensor, tile_size_pass2, tile_overlap)
        
        pass2_time = time.time() - pass2_start
        print(f"Pass 2 took: {pass2_time:.1f}s")
    
    total_upscale_time = time.time() - upscale_start
    print(f"Total upscaling took: {total_upscale_time:.1f}s")
    
    # Convert back to image
    output = output_tensor.squeeze(0).permute(1, 2, 0).float().cpu().numpy()
    output = (output * 255).clip(0, 255).astype(np.uint8)
    
    actual_upscaled_h, actual_upscaled_w = output.shape[:2]
    print(f"Upscaled size: {actual_upscaled_w}x{actual_upscaled_h}")
    
    # Resize to pre-crop dimensions using high-quality Lanczos
    print(f"Resizing to pre-crop size: {output_width}x{output_height}...")
    pil_img = Image.fromarray(output)
    pil_img = pil_img.resize((output_width, output_height), Image.LANCZOS)
    
    # Convert back to numpy (RGB)
    output_rgb = np.array(pil_img)
    
    # Save output in all requested formats
    print(f"\nSaving outputs ({', '.join(output_formats)})...")
    output_paths = save_image_formats(output_rgb, output_name, OUTPUT_DIR, output_formats)
    
    # Cleanup
    del model, img_tensor, output_tensor
    clear_memory()
    
    total_time = time.time() - start_time
    
    print(f"\n✅ Done!")
    print(f"Output size: {output_width}x{output_height}px")
    print(f"Equivalent print size: {scale_info['equivalent_print_size']} @ {dpi} DPI")
    
    if scale_info["crop_direction"] != "none":
        print(f"\n📋 CROP INFO FOR HUMAN:")
        print(f"   Direction: {scale_info['crop_direction'].upper()}")
        print(f"   Amount to crop: {scale_info['crop_amount_px']}px ({scale_info['crop_amount_inches']:.2f}\")")
        print(f"   After crop: {final_width}x{final_height}px = {target_width_inches}\" x {target_height_inches}\" @ {dpi} DPI")
    
    print(f"\nTotal time: {total_time:.1f}s")
    
    return {
        "output_paths": output_paths,
        "output_width": output_width,
        "output_height": output_height,
        "crop_direction": scale_info["crop_direction"],
        "crop_amount_px": scale_info["crop_amount_px"],
        "crop_amount_inches": scale_info["crop_amount_inches"],
        "final_width": final_width,
        "final_height": final_height,
        "equivalent_print_size": scale_info["equivalent_print_size"]
    }

print("✅ Functions defined!")

✅ Functions defined!


In [ ]:
# @title Run 6x Upscaling with Real-ESRGAN

print("=" * 60)
print("6X UPSCALING WITH REAL-ESRGAN")
print("=" * 60)

image_path = Path(INPUT_IMAGE)
if not image_path.exists():
    raise FileNotFoundError(f"Input file not found: {image_path}")

# Load image
print(f"\nProcessing: {image_path.name}")
img = cv2.imread(str(image_path))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]
target_w, target_h = w * 6, h * 6

print(f"Input size: {w}x{h}")
print(f"Target 6x size: {target_w}x{target_h}")

# Convert to tensor
img_tensor = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float() / 255.0
img_tensor = img_tensor.to(DEVICE)
if use_fp16 and DEVICE.type == "cuda":
    img_tensor = img_tensor.half()

# Load model
print(f"\nLoading model: {upscale_model}")
model_path = UPSCALE_MODELS_DIR / upscale_model
model = ModelLoader().load_from_file(str(model_path))
assert isinstance(model, ImageModelDescriptor), "Not an image model!"
model = model.to(DEVICE)
if use_fp16 and DEVICE.type == "cuda":
    model = model.half()
model.eval()

print(f"Model scale: {model.scale}x")

# Pass 1: 4x upscale
print("\nUpscaling (pass 1 of {})...".format(2 if USE_TWO_PASS else 1))
start_time = time.time()
output_tensor = upscale_with_tiles(model, img_tensor, tile_size, tile_overlap)
pass1_time = time.time() - start_time
p1_h, p1_w = output_tensor.shape[2], output_tensor.shape[3]
print(f"Pass 1 output: {p1_w}x{p1_h} (took {pass1_time:.1f}s)")

# Pass 2 if two-pass mode (16x total, then downscale to 6x)
if USE_TWO_PASS:
    print("Upscaling (pass 2 of 2)...")
    pass2_start = time.time()
    tile_size_pass2 = min(tile_size, 384)
    output_tensor = upscale_with_tiles(model, output_tensor, tile_size_pass2, tile_overlap)
    pass2_time = time.time() - pass2_start
    p2_h, p2_w = output_tensor.shape[2], output_tensor.shape[3]
    print(f"Pass 2 output: {p2_w}x{p2_h} (took {pass2_time:.1f}s)")

total_upscale_time = time.time() - start_time
print(f"Total upscaling took: {total_upscale_time:.1f}s")

# Convert to numpy - use chunked approach to avoid memory error
print("Converting to image...")
output_tensor = output_tensor.float()  # Ensure float32 for conversion
output_np = output_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
del output_tensor
gc.collect()
torch.cuda.empty_cache()

output_np = np.clip(output_np * 255, 0, 255).astype(np.uint8)

actual_h, actual_w = output_np.shape[:2]
print(f"Upscaled size: {actual_w}x{actual_h}")

# Lanczos resize to exact 6x
print(f"Resizing to 6x ({target_w}x{target_h}) with Lanczos...")
pil_img = Image.fromarray(output_np)
del output_np
gc.collect()

pil_img = pil_img.resize((target_w, target_h), Image.LANCZOS)
output_rgb = np.array(pil_img)

# Save
output_name = "9ab756e9-8505-48bd-aa38-3c360d1e29b6_real-esrgan-6x"
print(f"\nSaving outputs ({', '.join(OUTPUT_FORMATS)})...")
save_image_formats(output_rgb, output_name, OUTPUT_DIR, OUTPUT_FORMATS)

# Cleanup
del model, img_tensor, pil_img, output_rgb
clear_memory()

total_time = time.time() - start_time
print(f"\n✅ Done! Total time: {total_time:.1f}s")
print(f"Output: {target_w}x{target_h} (6x upscale, no downsizing)")

6X UPSCALING WITH REAL-ESRGAN

Processing: 9ab756e9-8505-48bd-aa38-3c360d1e29b6.jpg
Input size: 678x1024
Target 6x size: 4068x6144

Loading model: 4x-UltraSharp.pth
Model scale: 4x

Upscaling (pass 1 of 2)...
Processing 6 tiles (3x2)...
  Processed 6/6 tiles
Pass 1 output: 2712x4096 (took 12.2s)
Upscaling (pass 2 of 2)...
Processing 96 tiles (12x8)...
  Processed 56/96 tiles

## Single Image Processing (Optional)

Use the cell below if you want to process a single image with custom settings.

In [ ]:
# @title Single Image Upscale (Optional)

# Uncomment and modify to process a single image:

# single_result = upscale_and_resize(
#     image_path="input/your_image.jpg",
#     target_width=1650,   # 11 inches * 150 DPI
#     target_height=2100,  # 14 inches * 150 DPI
#     output_name="your_image_real-esrgan",
#     model_name="4x-UltraSharp.pth",
#     tile_size=512,
#     tile_overlap=32,
#     use_fp16=True
# )